# Machine Learning with scikit-learn

## Continuous Target Values

For a regression, we want a range of target values, not a binary category. Having one-hot encoded binary features with a target that is more ordinal than continuous is close to the worst case for using a regression. It's time to move away from the "Learning about Humans learning ML" dataset. We will use a well-known example dataset that is included with scikit-learn for this next section.

## Sample Datasets

A number of sample datasets are included with scikit-learn, either already bundled with a `load_*()` function for the smaller ones or with a `fetch_*()` function for the larger ones that can be obtained online. The `make_*()` functions create synthetic datasets with some randomness in their generation.

In [ ]:
from sklearn import datasets
[attr for attr in dir(datasets) if not attr.startswith('_')]

## California Housing Data

One downloadable dataset is the California housing data.  It has a target of house price, and 8 features.  There are 20 thousands samples in it, so it is reasonably large sized.  That is, it is nowhere close to the modern datasets of millions or billions of observations we sometimes work with; but it is also not a toy dataset of dozens or hundreds of observations that are often shown for demonstration purposes (including in prior lessons of this course).

In [ ]:
# To demonstrate downloading, remove cached version of dataset
!ls ~/scikit_learn_data
!rm ~/scikit_learn_data/*

In [ ]:
california = datasets.california_housing.fetch_california_housing()

In [ ]:
print(california.DESCR)

For convenience, it is often useful to massage the standard format of scikit-learn dataset objects.  These objects all have a attributes `.DESCR`, `.feature_names`, `.data`, `.target`.  The latter two are NumPy arrays.  The feature names are simply a Python list of strings.  The description is a single string that has newlines and simple formatting with citations and links inside it.

The California housing dataset has a fairly brief description.  For comparison, the Boston housing dataset has fewer observations but more citational detail.

In [ ]:
print(datasets.load_boston().DESCR)

In [ ]:
california.feature_names

In [ ]:
print("Features:", california.data.shape, california.data.dtype)
print("Target:", california.target.shape, california.target.dtype)

In [ ]:
# One DataFrame for everything
import pandas as pd
df_ca = pd.DataFrame(california.data, columns=california.feature_names)
df_ca['TARGET'] = california.target
df_ca.head()

We might wonder what the units are for the target variable of house price.  Consulting the [function documentation](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html) we can determine that is multiples of $100,000 (presumably 1997 dollars and prices, given citation).

Looking at some summary statistics is always worthwhile before we jump into our actual model.

In [ ]:
df_ca.describe()

## Comparing a Gaggle of Regressors

The DataFrame is useful for getting sense of the data, but for scikit-learn itself, we simply want to work with the `.data` and `.target` arrays.

In [ ]:
X = california.data
y = california.target

For validation—as usual—we want a train/test split.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

The metrics we use in the below code are `explained_variance_score`, `mean_absolute_error`, and `r2_score`. Many other metrics are are available, mostly within the `sklearn.metrics` submodule.

In [ ]:
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

The particular regressors we choose does not reflect any deep decision. Most are somewhat in the family of linear regression. RANSAC is tried because it is meant to be more resilient against outliers in data. This is sometimes more strongly predictive than generic linear regression. One of several Guassian techniques is shown as an example—it behaves worthlessly for this example, at least without hyperparameter tuning.

In [ ]:
r2_score?

In [ ]:
from time import time

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, RANSACRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.svm import SVR
from sklearn.svm import LinearSVR

regressors = [
    LinearRegression(), 
    RANSACRegressor(), 
    KNeighborsRegressor(),
    KNeighborsRegressor(n_neighbors=9, metric='manhattan'),
    SVR(),
    LinearSVR(),
    GaussianProcessRegressor(),
    SVR(kernel='linear'), # Cf. LinearSVR: much slower, might be better or worse: 
]

In [ ]:
head = 6
for model in regressors[:head]:
    start = time()
    model.fit(X_train, y_train)
    train_time = time() - start
    start = time()
    predictions = model.predict(X_test)
    predict_time = time()-start    
    print(model)
    print("\tTraining time: %0.3fs)" % train_time)
    print("\tPrediction time: %0.3fs" % predict_time)
    print("\tExplained variance:", explained_variance_score(y_test, predictions))
    print("\tMean absolute error:", mean_absolute_error(y_test, predictions))
    print("\tR2 score:", r2_score(y_test, predictions))
    print()

Two models that are very slow to train are omitted in the "live" output in this course.  Those outputs would be the following:

```
GaussianProcessRegressor(alpha=1e-10, copy_X_train=True, kernel=None,
             n_restarts_optimizer=0, normalize_y=False,
             optimizer='fmin_l_bfgs_b', random_state=None)
    Training time: 161.517s
    Prediction time: 4.571s
    Explained variance: -0.0199393545636
    Mean absolute error: 1.91320341763
    R2 score: -2.79445305731
    
SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
    Training time: 4458.012s
    Prediction time: 0.843s
    Explained variance: 0.527882057056
    Mean absolute error: 0.601893862699
    R2 score: 0.517431654021
```

Neither of these has an especially good R2 score for this particular data.  `GaussianProcessRegressor` is negative, in fact (which is very bad).  `SVR` with a linear kernel is moderately good, but not better than simple `LinearRegression`.  However, notice that even though training took well over an hour, prediction takes less than a second.  That is not the fasted predictor, but it is also not the slowest among those that train orders of magnitude faster.  If this model performed the best, it might be worth spending the one-time training cost, and then be able to regress sufficiently quickly.

## Linear Models

With high dimensionality, plain linear regression tends to perform surprisingly well.  Variants may add improvements, but this technique from general statistics, or analytic math, is quite good—notwithstanding that it basically pre-dates machine learning per se.  A simple linear regression basically assumes the following:

* as a feature value varies, the target value varies proportionally
  * responses to features are global
  * responses are strictly linear
* features are not co-linear

Even if these assumptions are not stricly true, the model may perform well.  One common strategy to mitigate variance from these assumptions is to *penalize* the weights (coefficients) assigned to each feature.  An "$l1$ penalty" is known as Lasso regression and will force some coefficients to zero.  An $l2$ penalty is known as Ridge regression and damps coefficients.

### Equations and Visualizations for Penalties

In more detail, I plain linear regression is this calculation:

$$ \text{min}_{w, b} \sum_i || w^\mathsf{T}x_i + b  - y_i||^2 $$

For Lasso regression (**L**east **a**bsolute **s**hrinkage **s**election **o**perator), the equation is:

$$ \text{min}_{w, b} \sum_i || w^\mathsf{T}x_i + b  - y_i||^2  + \alpha ||w||_1$$

For Ridge regression, the equation is:

$$ \text{min}_{w,b}  \sum_i || w^\mathsf{T}x_i + b  - y_i||^2  + \alpha ||w||_2^2$$ 

The second term in the right hand side of the equation below is the L2 regularization that is part of ridge regression ($\alpha$ is usually chosen between 0.01 and 100).  The idea in Ridge regression is to mutually minimize the error terms and the penalty constant.

It is probably easier to visualize these than look at the equations.  Ridge regression may be a good choice when there are colinear predictors in the `X` matrix.  In the simplest characterization, Ridge pulls the OLS (ordinary least square) estimators closer to zero (but not actually set them to exactly zero).

![Geometric Interpretation of Ridge Regression](img/ridge_regression_geomteric.png)

Contrasting Lasso and Ridge (or $l1$ versus $l2$) we can visualize the difference as below.  Notice that some coefficients are actually zeroes in the Lasso model.

![L1 versus L2 regions](img/L1_and_L2_balls.png)

Let's try this all out.

In [ ]:
from sklearn.linear_model import Lasso, Ridge

lr = LinearRegression()
lasso = Lasso()
ridge = Ridge()

for model in [lr, lasso, ridge]:
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    print(model)
    print("\tExplained variance:", explained_variance_score(y_test, predictions))
    print("\tMean absolute error:", mean_absolute_error(y_test, predictions))
    print("\tR2 score:", r2_score(y_test, predictions))
    print()

## Pitfalls of Linear Models

Famously, there can be many features of data that are not well captured in any (naïve) linear model. [Francis Anscombe](https://en.wikipedia.org/wiki/Frank_Anscombe) created his [Anscombe's quartet](https://en.wikipedia.org/wiki/Anscombe%27s_quartet) as an illustration of this. The several distributions in it have nearly the same or identical means and variances along X and Y axes, have almost the same correlations of X and Y, have the same linear regression lines and coefficient of determination. 

Let us look at some basic statistics and regressions on this data. Let us also look at the statistical properties of the elements of the quartet. In particular, notice the means and standard deviations (of both the x and y features), and the min/max and quartiles of the first three x feature collections.

This sample dataset is included in the Seaborn visualization library:

In [ ]:
import seaborn as sns
df = sns.load_dataset("anscombe")
df.pivot(columns='dataset').describe()

The overall data collection has similar (but not quite identical) properties to its subsects.

We might hope to tease apart what is going on with the several subsets by looking at the correlation of the features. Other than some floating point approximations, these are also identical.

In [ ]:
import numpy as np

df_1 = df[df.dataset=='I']
df_2 = df[df.dataset=='II']
df_3 = df[df.dataset=='III']
df_4 = df[df.dataset=='IV']

(np.corrcoef(df_1.x, df_1.y)[1,0],
 np.corrcoef(df_2.x, df_2.y)[1,0],
 np.corrcoef(df_3.x, df_3.y)[1,0],
 np.corrcoef(df_4.x, df_4.y)[1,0])

Actually visualizing the data gives us a very distinctly different impression of the data subsets.  The linear regressions are identical (under an [ordinary least squares fit](https://en.wikipedia.org/wiki/Ordinary_least_squares) here, but the point would be the same with a different fitting regime).

In [ ]:
%matplotlib inline
# Show the results of a linear regression within each dataset
sns.lmplot(x="x", y="y", col="dataset", hue="dataset", data=df,
           col_wrap=2, palette="muted", size=4,
           scatter_kws={"s": 50, "alpha": 1});

Humorously, Justin Matejka and George Fitzmaurice give a similar example with the [Datasaurus](https://www.autodeskresearch.com/publications/samestats) in _Same Stats, Different Graphs: Generating Datasets with Varied Appearance and Identical Statistics through Simulated Annealing_:

<img src="img/DataDino-600x455.gif" width="50%"/>

### The Moral of the Pitfalls

Obviously, the concerns raised by Anscombe hardly mean we should not use linear models.  In the examples we have looked at, basic OLS linear regression performed as well or better than all the other models we tried.  But there are certainly special cases where linear regression will be unable to distingish distributions whereas other techniques might be able to.

There is a tradeoff in these kinds of situations between two approaches.  One approach would be to use more feature engineering and general data cleanup before we get to our linear regression step.  We will think about those possibilities in later chapters.  Onother approach is to choose a different family of model from the start, and avoid the particular pitfalls the linear models have (but encounter other pitfalls in their place).

## Non-linear Regressors

Scikit-learn has a large number of additional regression models that are not based on linear models.  One relatively easy one to understand is `DecisionTreeRegressor`.  

The code below is adapted slightly from the [scikit-learn documentation](http://scikit-learn.org/stable/auto_examples/tree/plot_tree_regression.html#sphx-glr-auto-examples-tree-plot-tree-regression-py). Here we add an Epsilon-Support Vector Regression (`SVR`) regressor as well as a decision tree to show the contrast.

In the below one dimensional dataset, SVR produced a smoothed result that is much closer to the jittered sine curve that was used generate the points.  That is not really the point to the example though.  We want to get a visual sense of the kind of thing a decision tree is doing as a regressor rather than a classifier.  In higher dimensional cases, deciding continuous cut-points is often more effective than a smoothed kernel (and non-synthetic data isn't necessarily so smooth to start with). 

In [ ]:
%run src/decisiontree_regressor.py

We can see that `DecisionTreeRegressor` is conceptually similar to `DecisionTreeClassifier`.  At certain cut-points in the data (here just one dimension) a switch is made to a different target value.  However, rather than simply predict a flipped boolean value as in the prior lesson, we predict a new quantitative level for the target around cut points.

### Non-linearity in California Housing

Let us try a decision tree style against the multi-dimensional California housing data.  So far, a simple `LinearRegression` was the best we had done against this dataset.  As well as `DecisionTreeRegressor` of several different depths, we will add in `RandomForestRegressor`.  From [the documentation](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html):

> A random forest is a meta estimator that fits a number of classifying decision trees on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting. 

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

X_train, X_test, y_train, y_test = train_test_split(
        california.data, california.target, random_state=1)

regressors = [LinearRegression(),
              DecisionTreeRegressor(max_depth=5),
              DecisionTreeRegressor(max_depth=10),
              DecisionTreeRegressor(max_depth=20),
              RandomForestRegressor(max_depth=10),
              SVR(),]

for model in regressors:
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    print(model)
    print("\tExplained variance:", explained_variance_score(y_test, predictions))
    print("\tMean absolute error:", mean_absolute_error(y_test, predictions))
    print("\tR2 score:", r2_score(y_test, predictions))
    print()

At least three things to notice about this latest batch of comparisons:

* SVR (with an RBF kernel) does very poorly.  Smoothing around a kernel does not necessarily product best results.  Intuitively to me, I would expect this because housing prices tend to fall into plateaus around certain features (for example, houses in cities at particular latitude/longitude locations are characteristically spendy or cheap, depending on the city).
* The quality of a decision tree is sensitive to its depth.  The `max_depth` of 10 may or may not be optimal, but both higher and lower values are worse.
* As largely expected, a random forest improves on a decision tree.  Because cut points and their order are decided partially randomly, averaging a collection (10 by default) of these tends to produce a better result.  These algorithms are no among the expensive ones, but averaging ten decision trees will straightforwardly take 10x the time or cores as will just one tree.

## Next Lesson

**Clustering**: In the current lessson we explored the extended our knowlege of supervised learning to include regressors along with classifiers.  We payed special attential to linear models, which are very popular and often very successful; but we also contrast with some non-linear ones.  In the next lesson we will more on to unsupervised learning.

<a href="Clustering.ipynb"><img src="img/open-notebook.png" align="left"/></a>